In [ ]:

DATA_ROOT = "../../../data"
# split_name = "train"
# split_name = "dev"
split_name = "test"
prompt_data_path = f"{DATA_ROOT}/train-data-extract/process/{split_name}/concatenated_prefix_{split_name}.json"
label_data_path = f"{DATA_ROOT}/train-data-extract/process/{split_name}/concatenated_suffix_{split_name}.json"

import json
import numpy as np

# Load data
with open(prompt_data_path, 'r') as file:
    prompt_data = json.load(file)
with open(label_data_path, 'r') as file:
    label_data = json.load(file)

data = []
for prefix, suffix in zip(prompt_data, label_data):
    data.append(prefix['text'] + suffix['text'])

# data = data[:100]
data[0]

In [ ]:
from transformers import AutoTokenizer
from vllm import LLM
from vllm import SamplingParams

synthetic_model_name = "meta-llama/Llama-3.1-70B-Instruct"
# Initialize the model
tokenizer = AutoTokenizer.from_pretrained(synthetic_model_name)
llm = LLM(model=synthetic_model_name, tensor_parallel_size=4, max_model_len=4096)

In [ ]:
# def get_prompt(text):
#     return f"""
# Rewrite this entire text (all sentences with no exception) expressing the same meaning using different words. Aim to keep the rewriting similar in length to the original text. The text to be rewritten is identified as:
# ```
# {text}
# ```
# """.strip()

def get_prompt(text):
    return f"""
Your task is to rewrite the given text or code, maintaining the same meaning while using different words. Follow these guidelines:
- Preserve the original length as closely as possible.
- Ensure the rewritten version is clear and grammatically (and syntactically) correct.

The text to rewrite is enclosed below:
---
{text}
---
Please provide only the rewritten version without any additional comments or explanations.
""".strip()

# # Create a sampling params object.
sampling_params = SamplingParams(max_tokens=256, temperature=0.6, top_p=0.9, n=1)

prompt_text = [
    tokenizer.apply_chat_template([{"role": "user", "content": get_prompt(text)}], add_generation_prompt=True, tokenize=False)
    for text in data
]
outputs = llm.generate(prompt_text, sampling_params=sampling_params)
generated_text = [output.outputs[0].text for output in outputs]

In [ ]:
import json

# Combine the data into a list of dictionaries
output_data = [
    {
        "positive": text,
        "negative": paraphrase,
    }
    for text, paraphrase in zip(data, generated_text)
]

# Save the data to a JSON file
output_file_path = f"paraphrases.{split_name}.jsonl"
with open(output_file_path, 'w') as file:
    for line in output_data:
        file.write(json.dumps(line) + '\n')

print(f"Data saved to {output_file_path}")